In [ ]:
import os
import numpy as np
import math
import ogr
from osgeo import gdal, osr, gdalconst

#---------------------------Import satellite images-----------------------------------------------------------------------#


def importsatimage(imagename):
    
    imagenameM = imagename + "_M.tif"    
    print imagenameM
    satM = gdal.OpenShared(os.path.join('sixteen_band/',imagenameM),gdalconst.GA_Update)
    
        # Open the data source and read in the extent
    
    if os.path.isfile('sixteen_band/'+imagenameM):
        satM = gdal.OpenShared(os.path.join('sixteen_band/',imagenameM),gdalconst.GA_Update)
    else:
        raise IOError('Could not find file ' + imagenameM)
    if satM is None:
        raise IOError('Could open file ' + str(satM))
    

    
    return satM

#--------------------------Convert Geotiff files to numpy arrays, resize the arrays same size----------------------------#




def geotiff2numpy(sat, dir):

    L1 = np.array(sat.GetRasterBand(1).ReadAsArray())
    L2 = np.array(sat.GetRasterBand(2).ReadAsArray())
    L3 = np.array(sat.GetRasterBand(3).ReadAsArray())
    L4 = np.array(sat.GetRasterBand(4).ReadAsArray())
    L5 = np.array(sat.GetRasterBand(5).ReadAsArray())
    L6 = np.array(sat.GetRasterBand(6).ReadAsArray())
    L7 = np.array(sat.GetRasterBand(7).ReadAsArray())
    L8 = np.array(sat.GetRasterBand(8).ReadAsArray())
 


    #convert bands in XYZ format
    
    row,col = L1.shape
    R,C = np.mgrid[:row,:col]

    
    poop = np.full((3*5, 1), dir, dtype=np.dtype('a25'))
   
    
    L2T = L2.ravel()[np.newaxis].T
    L3T = L3.ravel()[np.newaxis].T
    L4T = L4.ravel()[np.newaxis].T
    L5T = L5.ravel()[np.newaxis].T
    L6T = L6.ravel()[np.newaxis].T
    L7T = L7.ravel()[np.newaxis].T
    L8T = L8.ravel()[np.newaxis].T
    name = np.full((row*col, 1), dir, dtype=np.dtype('a25'))


    XYL1 = np.column_stack((C.ravel(),R.ravel(), L1.ravel() ))
    XYLall = np.concatenate((name, XYL1, L2T, L3T, L4T, L5T, L6T, L7T, L8T),axis=1)

    print XYLall

    return XYLall
#

#----------------------------Import polygon files and reproject them-----------------------------------------------------#

def importpolygons(dir, file, M_col, M_row, x_max, y_min):

    polygonfilename = 'train_geojson_v3/' + dir + '/' + file

    # Define pixel_size and NoData value of new raster

    NoData_value = 255

    # Open the data source and read in the extent
    polygonfile = ogr.Open(polygonfilename)
    
    if os.path.isfile(polygonfilename):
        polygon = ogr.Open(polygonfilename)
    else:
        raise IOError('Could not find file ' + polygonfilename)
    if polygon is None:
        raise IOError('Could open file ' + str(polygon))
    
    polygon_layer = polygon.GetLayer()
    polygon_srs = polygon_layer.GetSpatialRef()
    x_min = 0
    y_max = 0
    
    if x_max == 999:
        x_min, x_max, y_min, y_max = polygon_layer.GetExtent()
    pixel_sizex = x_max/M_col
    pixel_sizey = y_min/M_row
    print x_min, x_max, y_min, y_max, pixel_sizex, pixel_sizey

    # Create the destination data source
    
    x_res = int((x_max - x_min) / pixel_sizex)
    y_res = -1*int((y_max - y_min) / pixel_sizey)
    target_ds = gdal.GetDriverByName('MEM').Create('', x_res, y_res, gdal.GDT_Byte)
    target_ds.SetGeoTransform((0, pixel_sizex, 0, 0, 0, pixel_sizey))
    band = target_ds.GetRasterBand(1)
    band.SetNoDataValue(NoData_value)

    # Rasterize
    
    gdal.RasterizeLayer(target_ds, [1], polygon_layer, burn_values=[1])
    print target_ds.GetGeoTransform() 
    print target_ds.RasterYSize
    # Read as array
    array = band.ReadAsArray()
    print array
    
    return array, x_max, y_min

#def rasterizepolygons():
#    return




#---------------------------------------------------------------------#


traindir ='./train_geojson_v3/'

for subdir, dirs, files in os.walk(traindir):

    for dir in dirs:
        
        ImgM = importsatimage(dir)
            
        M_row = ImgM.RasterYSize
        M_col = ImgM.RasterXSize

        data_matrix =geotiff2numpy(ImgM, dir)
        
        for subdir, dirs, files in os.walk('./train_geojson_v3/' + dir + '/'):
            
            files.sort(reverse=True)
            

            for file in files:
                if 'Grid' in file:
                    print 'Grid'
                    array, x_max, y_min = importpolygons(dir, file, M_col, M_row, 999, -999)
                elif '.directory' in file:
                    break
                else: 
                    #print dir, file, M_col, M_row, x_max, y_min
                    array, x, y = importpolygons(dir, file, M_col, M_row, x_max, y_min)
   



6070_2_3_M.tif
[['6070_2_3' '0' '0' ..., '255' '366' '310']
 ['6070_2_3' '1' '0' ..., '249' '350' '275']
 ['6070_2_3' '2' '0' ..., '293' '366' '345']
 ..., 
 ['6070_2_3' '832' '837' ..., '299' '491' '363']
 ['6070_2_3' '833' '837' ..., '299' '476' '359']
 ['6070_2_3' '834' '837' ..., '325' '516' '412']]
Grid
0.0 0.0090126 -0.009045 0.0 1.07935329341e-05 -1.07935560859e-05
(0.0, 1.0793532934131617e-05, 0.0, 0.0, 0.0, -1.0793556085918735e-05)
838
[[1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 ..., 
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]
 [1 1 1 ..., 1 1 1]]
0 0.0090126 -0.009045 0 1.07935329341e-05 -1.07935560859e-05
(0.0, 1.0793532934131617e-05, 0.0, 0.0, 0.0, -1.0793556085918735e-05)
838
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]
0 0.0090126 -0.009045 0 1.07935329341e-05 -1.07935560859e-05
(0.0, 1.0793532934131617e-05, 0.0, 0.0, 0.0, -1.0793556085918735e-05)
838
[[0 0 0 ..., 0 0 0]
 [0 0